In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# 난수 생성 방식을 고정
np.random.seed(100)
tf.random.set_seed(100)

df = pd.read_csv("data\\data_Advertising.csv")

print(df.head(5),'\n')

   Unnamed: 0     FB    TV  Newspaper  Sales
0           1  230.1  37.8       69.2   22.1
1           2   44.5  39.3       45.1   10.4
2           3   17.2  45.9       69.3    9.3
3           4  151.5  41.3       58.5   18.5
4           5  180.8  10.8       58.4   12.9 



In [2]:
df.drop(columns=['Unnamed: 0'])

,FB,TV,Newspaper,Sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9
...,...,...,...,...
195,38.2,3.7,13.8,7.6
196,94.2,4.9,8.1,9.7
197,177.0,9.3,6.4,12.8
198,283.6,42.0,66.2,25.5


In [3]:
X = df.drop(columns=['Sales'])
Y = df['Sales']

train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=0.3)

In [4]:
# Pandas의 DataFrame을 Tensorflow의 Dataset 형태로 변환
train_ds = tf.data.Dataset.from_tensor_slices((train_X.values, train_Y.values))

# shuffle로 섞어주고 Batch 나누기
train_ds = train_ds.shuffle(len(train_X)).batch(batch_size=5)

In [5]:
# take method로 batch 확인하기
[(train_features_batch, label_batch)] = train_ds.take(1)

In [6]:
print('\nFB, TV, Newspaper batch Data:\n',train_features_batch)
print('Sales batch Data:',label_batch)


FB, TV, Newspaper batch Data:
 tf.Tensor(
[[102.  296.4  36.3 100.9]
 [ 40.  228.   37.7  32. ]
 [ 79.    5.4  29.9   9.4]
 [  7.   57.5  32.8  23.5]
 [147.  240.1   7.3   8.7]], shape=(5, 4), dtype=float64)
Sales batch Data: tf.Tensor([23.8 21.5  5.3 11.8 13.2], shape=(5,), dtype=float64)


In [7]:
# Model 구현
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, input_shape=(4,)),
    tf.keras.layers.Dense(1),
])

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                50        
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 61
Trainable params: 61
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
model.compile(loss='mean_squared_error', optimizer='adam')
history = model.fit(train_ds, epochs=100, verbose=1)

Epoch 1/100
28/28 [==============================] - 0s 655us/step - loss: 7930.0054
Epoch 2/100
28/28 [==============================] - 0s 591us/step - loss: 2891.0146
Epoch 3/100
28/28 [==============================] - 0s 591us/step - loss: 1284.9661
Epoch 4/100
28/28 [==============================] - 0s 628us/step - loss: 805.7966
Epoch 5/100
28/28 [==============================] - 0s 591us/step - loss: 519.1624
Epoch 6/100
28/28 [==============================] - 0s 628us/step - loss: 333.0401
Epoch 7/100
28/28 [==============================] - 0s 574us/step - loss: 202.6297
Epoch 8/100
28/28 [==============================] - 0s 591us/step - loss: 126.4271
Epoch 9/100
28/28 [==============================] - 0s 591us/step - loss: 78.6902
Epoch 10/100
28/28 [==============================] - 0s 554us/step - loss: 52.3910
Epoch 11/100
28/28 [==============================] - 0s 628us/step - loss: 37.9799
Epoch 12/100
28/28 [==============================] - 0s 591us/step - loss

28/28 [==============================] - 0s 591us/step - loss: 3.9772
Epoch 100/100
28/28 [==============================] - 0s 591us/step - loss: 3.9045


In [9]:
loss = model.evaluate(test_X, test_Y)

2/2 [==============================] - 0s 2ms/step - loss: 2.4244


In [10]:
prediction = model.predict(test_X)

In [11]:
print("Test Data의 Loss값: ", loss)
for i in range(5):
    print("=======================================")
    print("%d Test Data Label: %f" % (i, test_Y.iloc[i]))
    print("%d Test Data Prediction: %f" % (i, prediction[i][0]))

Test Data의 Loss값:  2.4243876934051514
0 Test Data Label: 6.600000
0 Test Data Prediction: 10.833433
1 Test Data Label: 20.700000
1 Test Data Prediction: 20.372505
2 Test Data Label: 17.200000
2 Test Data Prediction: 17.448648
3 Test Data Label: 19.400000
3 Test Data Prediction: 19.980009
4 Test Data Label: 21.800000
4 Test Data Prediction: 21.679955
